In [6]:
import music21 
import os
from tqdm import tqdm
import numpy as np

In [ ]:
# base_dir = '../midi_songs'

# file = f'{base_dir}/FFIX_Piano.mid'

# score = music21.converter.parse(file).chordify() 

# print(score.show('text'))

In [2]:
def convert_parse(file_path): 
    try: 
        score = music21.converter.parse(file_path) 
    except: 
        score = None 
    return score

base_dir = '../midi_songs'

songlist = os.listdir(base_dir)
originalscores = [convert_parse(f'{base_dir}/{song}') for song in tqdm(songlist)] 
originalscores = [i for i in originalscores if i != None]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:07<00:00, 12.87it/s]


In [5]:
# originalscores

In [3]:
def monophonic(stream): 
    try: 
        length = len(music21.instrument.partitionByInstrument(stream).parts)
    except: 
        length = 0 
    if length == 1: 
        return length

originalscores = [song.chordify() for song in tqdm(originalscores)]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 90/90 [01:21<00:00,  1.11it/s]


In [7]:
# originalscores

In [ ]:
assert False

In [4]:
originalchords = [[] for i in originalscores] 
originaldurations = [[] for i in originalscores] 
originalkeys = []

#extract notes, chords, durations, and keys
for idx, song in enumerate(originalscores): 
    originalkeys.append(str(song.analyze('key')))
    
    for element in song: 
        if isinstance(element, music21.note.Note): 
            originalchords[idx].append(element.pitch)
            originalDurations[idx].append(element.duration.quarterLength)
        elif isinstance(element, music21.chord.Chord): 
            originalchords[idx].append('.'.join(str(n) for n in element.pitches))
            originaldurations[idx].append(element.duration.quarterLength)

In [10]:
#create a list of chords and durations from songs in C major 

# cMajorChords = [c for c,k in zip(originalchords, originalkeys) if k == 'C major']
# cMajorDurations = [c for (c,k) in zip(originaldurations, originalkeys) if k == 'C major']

In [8]:
#map chords to integers
uniquechords = np.unique([i for s in originalchords for i in s])
chordtoint = dict(zip(uniquechords, list(range(0, len(uniquechords)))))

#map unique durations to integers 
uniquedurations = np.unique([i for s in originaldurations for i in s])
durationstoint = dict(zip(uniquedurations, list(range(0, len(uniquedurations)))))

print(len(uniquechords))
print(len(uniquedurations))

10267
46


In [10]:
int_to_chord = {i:c for c, i in chordtoint.items()}
int_to_duration = {i:c for c, i in durationstoint.items()}



In [18]:
# Define sequence length
sequenceLength = 32

# Define empty arrays for train data
trainChords = []
trainDurations = []

# Construct training sequences for chords and durations
for s in range(len(originalchords)):
    chordList = [chordtoint[c] for c in originalchords[s]]
    durationList = [durationstoint[d] for d in originaldurations[s]]
    for i in range(len(chordList) - sequenceLength):
        trainChords.append(chordList[i:i+sequenceLength])
        trainDurations.append(durationList[i:i+sequenceLength])

In [19]:
from keras.utils import to_categorical

trainChords = trainChords[:501]
trainDurations = trainDurations[:501]
trainChords = to_categorical(trainChords).transpose(0,2,1)